# Frechet Inception Distance Calculation

Este notebook fue utilizado para calcular la distancia de Frechét-Inception, modificada para utilizar la red Lensfinder, utilizada para evaluar la calidad de las imágenes generadas

In [5]:
import sys
from pathlib import Path

# Agregar la carpeta src al Path para poder cargar los métodos y clases necesarias.
# También agregamos la carpeta data.
src_path = str(Path.cwd().parents[0] / "src")

if src_path not in sys.path:
    sys.path.append(src_path)

In [6]:
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Cargar librerías 

In [13]:
import numpy as np
from scipy.linalg import sqrtm
from keras import backend as K
from keras.models import Model, load_model
from experiments import read_experiment_dataset

## Frechet Inception Distance

In [3]:
def preprocess_images(images):
    MEAN = 4.3368545e-13 #precalculated from training dataset
    STD = 5.5039e-12 #precalculated from training dataset
    images[images == 100] = 0.0
    return (images - MEAN) / STD

In [4]:
def frechet_lensfinder_distance(model,real_images,synthetic_images):
    #preprocess datasets
    real_images_scaled = preprocess_images(real_images)
    synthetic_images_scaled = preprocess_images(synthetic_images)
    
    #get activations
    real_activations = model.predict(real_images_scaled)
    synthetic_activations = model.predict(synthetic_images_scaled)
    
    #Calculate score
    real_mean = real_activations.mean(axis=0)
    synthetic_mean = synthetic_activations.mean(axis=0)
    real_cov = np.cov(real_activations,rowvar=False)
    synthetic_cov = np.cov(synthetic_activations,rowvar=False)
    
    #Trace elements
    squared_cov_dot, error = sqrtm(
        synthetic_cov.dot(real_cov),
        disp=False
    )
    if np.iscomplexobj(squared_cov_dot):
        squared_cov_dot = squared_cov_dot.real
    trace = np.trace(synthetic_cov + real_cov - 2*squared_cov_dot)
    
    return np.sum((synthetic_mean - real_mean)**2) + trace

## Calculo del FID

In [10]:
real_lens, df_real_lens = read_experiment_dataset('../data/experiments/real_lens_100_images')
synthetic_lens, df_synthetic_lens = read_experiment_dataset('../data/experiments/50_epochs_100_images')

In [14]:
#Read classifier model
def custom_relu(x):
    relu = K.relu(x)
    return (relu - 0.3989422804014327) * 1.712858550449663

lensfinder_model = load_model('../models/lensfinder/lensfinder_model_val_loss_0.25.hdf5',custom_objects={'custom_relu': custom_relu})
lensfinder_model = Model(inputs=lensfinder_model.inputs, outputs=lensfinder_model.layers[-2].output)

In [15]:
frechet_lensfinder_distance(lensfinder_model,real_lens,synthetic_lens)

40.27473739979382